In [2]:
import pomegranate
import pandas as pd
import os 
import numpy as np

In [6]:
# set working directory (Change for youe)
os.chdir('C:/Users/ander/Google Drive/Columbia/Fall 2019/Capstone/Dotin-Columbia-Castone-Team-Alpha-')
data = pd.read_csv('Models/Q1_Mouse Activity/Data/direction_data.csv')
data.head()

,Unnamed: 0,User Id,Distance X,Distance Y,Direction
0,0,365.0,601.0,167.0,North East
1,1,365.0,1.0,-1.0,North West
2,2,365.0,0.0,0.0,No Movement
3,3,365.0,0.0,0.0,No Movement
4,4,365.0,0.0,0.0,No Movement


In [7]:
direction = data['Direction']

In [9]:
proportion = pd.DataFrame(data['Direction'].value_counts())
proportion['Proportion'] = proportion['Direction'] /proportion['Direction'].sum()
proportion

,Direction,Proportion
No Movement,1551783,0.191068
North,1043138,0.128439
East,959875,0.118187
North East,935212,0.115151
South East,845577,0.104114
West,823033,0.101338
South West,769229,0.094714
South,612332,0.075395
North West,581457,0.071594


In [11]:
states_ = pd.unique(direction)
states_

array(['North East', 'North West', 'No Movement', 'North', 'South East',
       'South West', 'South', 'West', 'East'], dtype=object)

In [13]:
user_direction = data[['User Id', 'Direction']]
flatten_data = user_direction.groupby('User Id', as_index=False).agg(list)
flatten_data

,User Id,Direction
0,365.0,"[North East, North West, No Movement, No Movem..."
1,371.0,"[North East, North West, South, South, North W..."
2,373.0,"[North West, South West, North, North, North, ..."
3,374.0,"[North East, No Movement, No Movement, No Move..."
4,375.0,"[South West, South West, South West, South, So..."
...,...,...
725,1834.0,"[North East, No Movement, South East, North, S..."
726,1835.0,"[North East, North West, North West, North Wes..."
727,1836.0,"[North East, South West, South, South, South, ..."
728,1837.0,"[East, North East, North East, North East, Nor..."
